In [19]:
import models
import pandas as pd
from sklearn.pipeline import make_pipeline
from preprocess import TypeConverter
from splitter import StratifiedRegressionSplit
from sklearn.metrics import r2_score

In [20]:
variables = pd.read_excel('/root/workspace/clanci/realestate_prices/data/covariates.xlsx')
data = pd.read_excel('/root/workspace/clanci/realestate_prices/data/hp_ljubljana_new_with_rooms.xlsx')

variables = variables.to_dict(orient='records')
target = []
features = []
for feature in variables:
    feature['type'] = int if feature['type'] == 'int' else float if feature['type'] == 'float' else str
    if feature['group'] == 'target':
        target = feature
        continue
    else:
        features.append(feature)

X = data[[feature['name'] for feature in features]]
y = data[target['name']]

# PREPROCESS PIPLINE
preprocc_pip = make_pipeline(TypeConverter(variables=features))
preprocc_pip.fit(X)
X = preprocc_pip.transform(X)

models.models['Polynomial Regression']['preprocessing_pipline'].fit(X)
X = models.models['Polynomial Regression']['preprocessing_pipline'].transform(X)

/tmp/ipykernel_8169/2312949821.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  data = pd.read_excel('/root/workspace/clanci/realestate_prices/data/hp_ljubljana_new_with_rooms.xlsx')


In [21]:
cv=StratifiedRegressionSplit(n_splits=10, n_bins = 10, test_size=0.3, random_state=0)

for train, test in cv.split(X,y):
    model = models.models['Polynomial Regression']['fitting_pipline'].estimator
    model.fit(X.loc[train,:], y[train])
    y_pred = model.predict(X.loc[test,:])
    score = r2_print(score)score(y[test], y_pred)
    print(score)
    

-7.608955858100643e+25
-5.176809156571062e+22
-8.611363475902049e+22
-2.495284359637199e+23
-6.622818755830587e+23
-2.044663727436949e+23
-3.7506061240816745e+24
-7.077646858835183e+25
-9.029537842793505e+22
-3.0819308831710436e+23
